In [72]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import statsmodels.api as sm
import plotly.graph_objects as go

In [56]:
# Load sugarscape model csv data and ignore the first 6 rows
df = pd.read_csv(os.path.join('data', 'Sugarscape 2 Constant Growback meta vs growth-table.csv'), skiprows=6)

# Remove any rows where columns "max-vis >= min-vis" and "max-meta >= min-meta" are the string "False"
df = df[(df['max-vis >= min-vis'] == True) & (df['max-meta >= min-meta'] == True)]

# Remove unnecessary columns
df = df.drop(columns=['max-vis >= min-vis', 'max-meta >= min-meta', 'visualization',
                       '[run number]', 'initial-population', '[step]', 'max-vis', 'min-vis'])

# Rename columns
df = df.rename(columns={'mean [metabolism] of turtles:': 'mean_metabolism',
                        'mean [vision] of turtles:': 'mean_vision',
                        'count turtles': 'turtles_count'})

# Convert all columns to numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Remove rows with NaN
df = df.dropna()

In [86]:
# Create a new df where the rows are grouped by max-meta, min-meta, and sugar-growth-rate
# grouped_df = df.groupby(['max-meta', 'min-meta', 'sugar-growback-rate']).mean().reset_index()

# Group by median
grouped_df = df.groupby(['max-meta', 'min-meta', 'sugar-growback-rate']).median().reset_index()

In [87]:
# Add a new column (range of metabolism) to the grouped df
grouped_df['range_of_meta'] = grouped_df['max-meta'] - grouped_df['min-meta']

In [88]:
grouped_df

,max-meta,min-meta,sugar-growback-rate,turtles_count,mean [metabolism] of turtles,mean [vision] of turtles,range_of_meta
0,1,1,0.25,420.0,1.000000,3.901545,0
1,1,1,0.50,472.0,1.000000,3.581663,0
2,1,1,0.75,472.0,1.000000,3.584473,0
3,1,1,1.00,469.0,1.000000,3.584756,0
4,1,1,1.25,471.0,1.000000,3.587670,0
...,...,...,...,...,...,...,...
75,10,1,1.00,122.0,1.908333,3.714359,9
76,10,1,1.25,124.0,1.907935,3.705312,9
77,10,1,1.50,124.5,1.894523,3.683259,9
78,10,1,1.75,123.0,1.905512,3.682363,9


In [89]:
from scipy.interpolate import griddata

# Assuming grouped_df contains your data

# Define grid
x = grouped_df['sugar-growback-rate']
y = grouped_df['range_of_meta']
z = grouped_df['turtles_count']
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# Interpolate
zi = griddata((x, y), z, (xi, yi), method='linear')

# Create surface plot
fig = go.Figure(data=[go.Surface(x=xi, y=yi, z=zi)])

# Update layout
fig.update_layout(scene=dict(
                    xaxis_title='Sugar Growback Rate',
                    yaxis_title='Range of Meta',
                    zaxis_title='Turtles Count'))

fig.show()

In [90]:
# Make another plot like the one above, but instead of points created a surface plot
fig = px.scatter_3d(grouped_df, x='sugar-growback-rate', y='range_of_meta', z='turtles_count')
fig.update_traces(mode='markers')
fig.add_traces(px.scatter_3d(grouped_df, x='sugar-growback-rate', y='range_of_meta', z='turtles_count').data)
fig.update_traces(marker=dict(size=5))
fig.update_traces(mode='markers', marker=dict(line=dict(width=2, color='DarkSlateGrey')))
fig.update_layout(scene = dict(
                    xaxis_title='sugar-growback-rate',
                    yaxis_title='range_of_meta',
                    zaxis_title='turtles_count'))
fig.show()

In [91]:
# Do a linear regression on the data using statsmodels
X = grouped_df[['sugar-growback-rate', 'range_of_meta']]
y = grouped_df['turtles_count']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          turtles_count   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.860
Method:                 Least Squares   F-statistic:                     243.4
Date:                Sun, 25 Feb 2024   Prob (F-statistic):           5.17e-34
Time:                        22:07:16   Log-Likelihood:                -416.23
No. Observations:                  80   AIC:                             838.5
Df Residuals:                      77   BIC:                             845.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 359.4207     13.556     26.514      0.000     332.427     386.414
sugar-growback-rate    43.6119      8.752      4.983      0.000      26.185      61.039
range_of_meta         -37.5117      1.745    -21.492      0.000     -40.987     -34.036
==============================================================================
Omnibus:                        1.249   Durbin-Watson:                   0.618
Prob(Omnibus):                  0.535   Jarque-Bera (JB):                1.054
Skew:                          -0.020   Prob(JB):                        0.590
Kurtosis:                       2.439   Cond. No.                         16.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""